In [1]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.python.platform
import matplotlib.image as mpimg
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [3]:
COLAB = True
BRANCH_NAME = 'hugo' # Change with the name of you toying branch, or master
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
GENERATE_PREDICTION = False #If True, will generate a CSV to submit on AICrowd

PREDICTIONS_SAVE_DIR = 'predictions/'
MODELS_SAVE_DIR = 'model_save/'
MODEL_NAME = 'cnn_handmade' #Chose between cnn_handmade and u-net

TRAINING_SIZE = 80 # Size of the training set, integer between 0 and 100, the remaining part is for testing
VALIDATION_SIZE = 0.20  # Size of the validation set, float between 0 and 1
SEED = 66478  # Set to None for random seed.
NUM_EPOCHS = 15

NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
BATCH_SIZE = 16  # 64
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [4]:
if COLAB:
    from helpers.colab import mount_and_pull
    mount_and_pull(BRANCH_NAME)

SyntaxError: invalid syntax (colab.py, line 15)

In [4]:
#TODO move into a python file for Unet model
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = layers.UpSampling2D((2, 2))(x)
    concat = layers.Concatenate()([us, skip])
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [5]:
#TODO move into a python file for Unet model
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = layers.Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    a0 = layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    a1 = layers.Flatten()(a0)
    a2 = layers.Dense(64, activation='relu')(a1)
    outputs = layers.Dense(2, activation='relu')(a2)
    model = models.Model(inputs, outputs)
    return model

In [6]:
def CNN() : 
    # Create the model
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(16, 16, 3), padding='same', use_bias=True))
    cnn_model.add(layers.MaxPooling2D((2, 2)))
    cnn_model.add(layers.Conv2D(128, (5, 5), activation='relu', input_shape=(8, 8, 3), padding='same', use_bias=True))
    cnn_model.add(layers.MaxPooling2D((2, 2)))
    cnn_model.add(layers.Flatten())
    cnn_model.add(layers.Dense(128, activation='relu'))
    cnn_model.add(layers.Dense(2))
    return cnn_model

In [7]:
models_dict = {'cnn_handmade' : {
        'name' : 'cnn',
        'model' : CNN,
        'save_dir' : MODELS_SAVE_DIR + 'cnn_handmade/'
    },
        'u-net' : {
        'name' : 'u-net',
        'model' : UNet,
        'save_dir' : MODELS_SAVE_DIR + 'u-net/'
    }}

In [8]:
model_chosen = models_dict[MODEL_NAME]

In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [11]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract patches into numpy arrays. Dim: (nb patch, 16, 16, 3)
training_data = extract_data(train_data_filename, 100)
training_labels = extract_labels(train_labels_filename, 100)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 132.63it/s]


Loaded 100 training images


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 162.69it/s]


Loaded 100 groudtruth images


In [16]:
#This cell is required for me (Ben), otherwise I get "convolution algorithm not found"
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [17]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [18]:
X_train, X_test, y_train, y_test = train_test_split(training_data, training_labels,\
                                                    train_size= TRAINING_SIZE/100, random_state=SEED)

print(X_train.shape)
print(X_test.shape)

(50000, 16, 16, 3)
(12500, 16, 16, 3)


In [19]:
num_epochs = NUM_EPOCHS

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(X_train.shape)
X_train = X_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 37053 c1 = 12947
Balancing training data...
25894
(50000, 16, 16, 3)
Number of data points per class: c0 = 12947 c1 = 12947


In [20]:
if RESTORE_MODEL:
    # It can be used to reconstruct the model identically.
    model = models.load_model(model_chosen['save_dir'])
else : 
    model = model_chosen['model']()
    with tf.device('/device:GPU:0'):
        model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        history = model.fit(X_train, y_train, epochs = NUM_EPOCHS ,validation_split=VALIDATION_SIZE)

Epoch 1/15
648/648 [==============================] - 19s 29ms/step - loss: 0.6133 - accuracy: 0.6396 - val_loss: 0.8105 - val_accuracy: 0.5016
Epoch 2/15
648/648 [==============================] - 23s 35ms/step - loss: 0.5716 - accuracy: 0.6781 - val_loss: 0.8060 - val_accuracy: 0.5003
Epoch 3/15
648/648 [==============================] - 24s 37ms/step - loss: 0.5526 - accuracy: 0.6960 - val_loss: 0.7322 - val_accuracy: 0.5766
Epoch 4/15
648/648 [==============================] - 23s 35ms/step - loss: 0.5271 - accuracy: 0.7221 - val_loss: 0.6641 - val_accuracy: 0.6333
Epoch 5/15
648/648 [==============================] - 27s 41ms/step - loss: 0.4940 - accuracy: 0.7481 - val_loss: 0.8709 - val_accuracy: 0.5098
Epoch 6/15
648/648 [==============================] - 30s 46ms/step - loss: 0.4575 - accuracy: 0.7754 - val_loss: 0.8253 - val_accuracy: 0.5783
Epoch 7/15
648/648 [==============================] - 31s 48ms/step - loss: 0.4172 - accuracy: 0.7976 - val_loss: 0.5908 - val_accuracy:

In [17]:
if not RESTORE_MODEL:
    model.save(model_chosen['save_dir'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_save/cnn_handmade/assets


In [21]:
pred_train = model.predict(X_train)
print(pred_train)
print("Training error rate: {:.2f}%".format(error_rate(pred_train, y_train)))

[[ 0.49630305 -0.50646657]
 [ 3.0319185  -3.2845793 ]
 [ 1.7245344  -2.5284963 ]
 ...
 [-0.65732044  0.7350807 ]
 [-0.3323874   0.34828112]
 [-6.2602987   5.439677  ]]
Training error rate: 11.95%


In [22]:
pred_test = model.predict(X_test)
print("Test error rate: {:.2f}%".format(error_rate(pred_test, y_test)))

Test error rate: 24.41%


In [23]:
F1_score(y_test, pred_test)

0.6070830650354152

In [24]:
print("Running prediction on training set")
prediction_training_dir = PREDICTIONS_SAVE_DIR + "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE + 1):
    pimg = get_prediction_with_groundtruth(model, train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(model, train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'predictions/predictions_training/'

In [23]:
if GENERATE_PREDICTION :
    predict_test_masks(model)
    masks_to_submission("submission.csv", PREDICTIONS_SAVE_DIR+"predictions_test/")

In [5]:
# downloading the resulting model from Colab
if COLAB:
    from helpers.colab import download_model
    download_model(MODEL_NAME, model_chosen['save_dir'])

SyntaxError: invalid syntax (colab.py, line 15)